In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time

# Setup Chrome
options = webdriver.ChromeOptions()
options.add_argument("--headless")

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)

driver.get("https://www.cdc.gov/health-topics.html")
time.sleep(5)  # wait for JS

# Parse rendered HTML with BeautifulSoup
soup = BeautifulSoup(driver.page_source, "html.parser")

data = []

for a in soup.select(".cdc-az-list .char-block ul li a"):
    name = a.get_text(strip=True)
    link = a.get("href")

    if name and link:
        if link.startswith("/"):
            link = "https://www.cdc.gov" + link

        data.append({
            "disease_name": name,
            "url": link
        })

driver.quit()

df = pd.DataFrame(data)
print(df.head())
print("Total:", len(df))

df.to_csv("cdc_health_topics.csv", index=False)


                                        disease_name  \
0                              Abortion Surveillance   
1  Abuse, Child — see Child Abuse and Neglect Pre...   
2                     Abuse, Elder — see Elder Abuse   
3                                Abusive Head Trauma   
4                              AcanthamoebaInfection   

                                                 url  
0  https://www.cdc.gov/reproductive-health/data-s...  
1  https://www.cdc.gov/child-abuse-neglect/about/...  
2   https://www.cdc.gov/elder-abuse/about/index.html  
3  https://www.cdc.gov/child-abuse-neglect/about/...  
4            https://www.cdc.gov/acanthamoeba/about/  
Total: 1714
